<a href="https://colab.research.google.com/github/itisEndymion/DARC/blob/main/CMT318_Lab_3_NGram_Language_Models_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMT318: Lab 3 - NGram Language Models Part 2 (Autumn 2024)

This notebook has been adapted from [UW's CSE 447 / CSE M 547 Project 1](https://drive.google.com/file/d/1Q1UD-vawgU-GfnvynWkV6nQD_3SizPho/view).

**IMPORTANT: Save a copy of the notebook before working on it. You can also download it if you prefer**

## Dataset

The data you need for this lab is available in Learning Central.

You are provided with three data files (a subset of the One Billion Word Language Modeling Benchmark). Each line in each file contains a whitespace-tokenized sentence.

- **1bbenchmark.train.tokens:** data for training your language models.
- **1bbenchmark.dev.tokens:** data for debugging and choosing the best hyperparameters.
- **1bbenchmark.test.tokens:** data for evaluating your language models.

**IMPORTANT:**
You will primarily use the development/validation dataset as the previously unseen data while (i) developing and testing your code, (ii) trying out different model and training design decisions, (iii) tuning the hyperparameters, and (iv) performing error analysis.
For scientific integrity, it is extremely important that you **use the test data only once**, just before you report all the final results. Otherwise, you will start overfitting on the test set indirectly.

## Exercise 0: The `nltk.lm` package

The Natural Language Toolkit includes a package specifically designed for n-gram language modeling: [nltk.lm](https://www.nltk.org/api/nltk.lm.html). Go through the tutorial included in the documentation to test some the functionalities offered by this package.

**IMPORTANT:** If you wish, you are allowed to use functions from `nltk.lm` in the reminder of this lab.

In [1]:
import nltk
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends



## Exercise 1: Build Language Models
You will build **unigram**, **bigram**, and **trigram** language models. To handle
out-of-vocabulary (OOV) words, **convert tokens that occur less than three times in the training data into a special `<unk>` token during training**. Remember to also add the `<s>` and `</s>` tokens.

You should have (at least) three functions called `unigram()`, `bigram()`, and `trigram()` that contain your implementations of these models, respectively. Feel free to add additional helper functions for better organization and/or debugging, but the above functions are required.

In [2]:
import os
from collections import Counter

def load_sentences(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip().split() for line in f.readlines()]


def handle_oov(sentences, min_count=3):
    word_counts = Counter(word for sentence in sentences for word in sentence)
    vocab = {word for word, count in word_counts.items() if count >= min_count}

    processed = [
        [word if word in vocab else "<unk>" for word in sentence]
        for sentence in sentences
    ]
    return processed, vocab


def train_model(sentences, n):
    train_data, vocab = padded_everygram_pipeline(n, sentences)
    model = MLE(n)
    model.fit(train_data, vocab)
    return model

## Exercise 2: Evaluate Your Models with Perplexity
Evaluate all your models using perplexity in two ways: (1) one without any smoothing technique; and (2) one with Laplace smoothing (also called add-one smoothing).

Report the unsmoothed and smoothed with Laplace smoothing version of perplexity scores of the unigram, bigram, and trigram language models for your training, development, and test sets.

In [3]:
import nltk
from nltk.lm import MLE, Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.util import ngrams
from collections import Counter

def load_sentences(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip().split() for line in f.readlines()]

def handle_oov(sentences, min_count=3):
    word_counts = Counter(word for sentence in sentences for word in sentence)
    vocab = {word for word, count in word_counts.items() if count >= min_count}
    processed = [
        [word if word in vocab else "<unk>" for word in sentence]
        for sentence in sentences
    ]
    return processed, vocab

def train_mle_model(sentences, n):
    train_data, vocab = padded_everygram_pipeline(n, sentences)
    model = MLE(n)
    model.fit(train_data, vocab)
    return model

def train_laplace_model(sentences, n):
    train_data, vocab = padded_everygram_pipeline(n, sentences)
    model = Laplace(n)
    model.fit(train_data, vocab)
    return model

def evaluate_perplexity(model, data_sentences, n):
    test_ngrams = [list(ngrams(pad_both_ends(sentence, n), n)) for sentence in data_sentences]
    all_ngrams = [ng for sent in test_ngrams for ng in sent]
    return model.perplexity(all_ngrams)

def evaluate_all(train_path, dev_path, test_path):
    n_values = [1, 2, 3]
    models = {'MLE': train_mle_model, 'Laplace': train_laplace_model}
    datasets = {
        'train': load_sentences(train_path),
        'dev': load_sentences(dev_path),
        'test': load_sentences(test_path)
    }

    train_sentences_raw = datasets['train']
    train_sentences, vocab = handle_oov(train_sentences_raw)

    datasets = {
        name: [[word if word in vocab else "<unk>" for word in sentence] for sentence in sents]
        for name, sents in datasets.items()
    }

    for n in n_values:
        print(f"\n{n}-gram Models ")
        for model_name, train_fn in models.items():
            model = train_fn(train_sentences, n)
            for set_name in ['train', 'dev', 'test']:
                ppl = evaluate_perplexity(model, datasets[set_name], n)
                print(f"{model_name} {n}-gram - {set_name} perplexity: {ppl:.2f}")

# Run evaluation
evaluate_all(
    train_path="1b_benchmark.train.tokens",
    dev_path="1b_benchmark.dev.tokens",
    test_path="1b_benchmark.test.tokens"
)



1-gram Models 
MLE 1-gram - train perplexity: 1083.22
MLE 1-gram - dev perplexity: 985.98
MLE 1-gram - test perplexity: 991.53
Laplace 1-gram - train perplexity: 1084.32
Laplace 1-gram - dev perplexity: 988.44
Laplace 1-gram - test perplexity: 993.89

2-gram Models 
MLE 2-gram - train perplexity: 77.07
MLE 2-gram - dev perplexity: inf
MLE 2-gram - test perplexity: inf
Laplace 2-gram - train perplexity: 1442.39
Laplace 2-gram - dev perplexity: 1669.75
Laplace 2-gram - test perplexity: 1665.48

3-gram Models 
MLE 3-gram - train perplexity: 7.30
MLE 3-gram - dev perplexity: inf
MLE 3-gram - test perplexity: inf
Laplace 3-gram - train perplexity: 4633.13
Laplace 3-gram - dev perplexity: 7071.58
Laplace 3-gram - test perplexity: 7040.14


## Exercise 3: Interpolation
To make your language model work better, you will implement **linear interpolation** smoothing between unigram, bigram, and trigram models. You should use the development data to choose the best values for the hyperparameters: $\lambda_1, \lambda_2, \lambda_3$, for unigram, bigram and trigram, respectively. For this lab, you should try a few combinations to find reasonable values, but remember that $\lambda_1 + \lambda_2 + \lambda_3 = 1$

We ask you to:

1. Experiment and report perplexity scores on training and development sets for five sets of values of $\lambda_1, \lambda_2, \lambda_3$ that you
tried, along with short descriptions of the strategies that you used to find better hyperparameters.
2. Report the training and development perplexity for the values $\lambda_1 = 0.1, \lambda_2 = 0.3, \lambda_3 = 0.6$.
3. Report perplexity on the test set, using the best combination of
hyperparameters that you chose from the development set. Specify those hyperparameters.

In [4]:
import math

def load_sentences(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip().split() for line in f.readlines()]

def handle_oov(sentences, min_count=3):
    word_counts = Counter(word for sentence in sentences for word in sentence)
    vocab = {word for word, count in word_counts.items() if count >= min_count}
    processed = [
        [word if word in vocab else "<unk>" for word in sentence]
        for sentence in sentences
    ]
    return processed, vocab

def train_mle_model(sentences, n):
    data, vocab = padded_everygram_pipeline(n, sentences)
    model = MLE(n)
    model.fit(data, vocab)
    return model

def linear_interpolated_score(word, context, models, lambdas):
    n = len(context) + 1
    total = 0.0
    for i, lm in enumerate(models):
        if i + 1 > n:
            continue
        sub_context = context[-(i):] if i > 0 else []
        total += lambdas[i] * lm.score(word, sub_context)
    return total

def evaluate_interpolated_perplexity(models, data_sentences, lambdas, n=3):
    total_log_prob = 0.0
    total_words = 0
    epsilon = 1e-10  # To prevent log(0)

    for sentence in data_sentences:
        sentence = list(pad_both_ends(sentence, n))
        for i in range(n - 1, len(sentence)):
            context = sentence[i - (n - 1):i]
            word = sentence[i]
            prob = linear_interpolated_score(word, context, models, lambdas)
            prob = max(prob, epsilon)  # Replace zero with small positive value
            total_log_prob += -math.log(prob, 2)
            total_words += 1

    return 2 ** (total_log_prob / total_words)


train_raw = load_sentences("1b_benchmark.train.tokens")
dev_raw = load_sentences("1b_benchmark.dev.tokens")
test_raw = load_sentences("1b_benchmark.test.tokens")

train_data, vocab = handle_oov(train_raw)
dev_data = [[w if w in vocab else "<unk>" for w in s] for s in dev_raw]
test_data = [[w if w in vocab else "<unk>" for w in s] for s in test_raw]

# Train n-gram models
uni = train_mle_model(train_data, 1)
bi = train_mle_model(train_data, 2)
tri = train_mle_model(train_data, 3)

models = [uni, bi, tri]
lambdas = [0.1, 0.3, 0.6]

# Evaluate
print("Train perplexity:", evaluate_interpolated_perplexity(models, train_data, lambdas))
print("Dev perplexity:", evaluate_interpolated_perplexity(models, dev_data, lambdas))

Train perplexity: 10.401568218017589
Dev perplexity: 287.1731994415753


## Exercise 4: Analysis

Provide answers to the following questions supporting them with appropriate empirical experimental evidence:

1. If you use half of the training data, would it increase or decrease the perplexity of previously unseen data? Why?
2. If you convert all tokens that appeared less than five times to `<unk>` (a special symbol for out-of-vocabulary tokens), would it increase or decrease the perplexity on the previously unseen data compared to an approach that converts only those words that appeared just once to `<unk>`? Why?

1.It would increase the perplexity on unseen data. Because reducing the amount of training data increases perplexity, due to poorer n-gram coverage.


In [8]:
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.util import ngrams
from collections import Counter

def load_sentences(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip().split() for line in f]

def handle_oov(sentences, min_count=3):
    word_counts = Counter(word for sent in sentences for word in sent)
    vocab = {w for w, c in word_counts.items() if c >= min_count}
    processed = [[w if w in vocab else "<unk>" for w in sent] for sent in sentences]
    return processed, vocab

def train_laplace_model(sentences, n):
    train_data, vocab = padded_everygram_pipeline(n, sentences)
    model = Laplace(n)
    model.fit(train_data, vocab)
    return model

def evaluate_perplexity(model, data_sentences, n):
    test_ngrams = [list(ngrams(pad_both_ends(sent, n), n)) for sent in data_sentences]
    all_ngrams = [ng for sent in test_ngrams for ng in sent]
    return model.perplexity(all_ngrams)

def question1_half_training(train_path, dev_path):
    full_raw = load_sentences(train_path)
    half_raw = full_raw[: len(full_raw) // 2]
    dev_raw = load_sentences(dev_path)

    full_proc, vocab = handle_oov(full_raw)
    half_proc = [[w if w in vocab else "<unk>" for w in s] for s in half_raw]
    dev_proc = [[w if w in vocab else "<unk>" for w in s] for s in dev_raw]

    full_model = train_laplace_model(full_proc, 3)
    half_model = train_laplace_model(half_proc, 3)

    ppl_full = evaluate_perplexity(full_model, dev_proc, 3)
    ppl_half = evaluate_perplexity(half_model, dev_proc, 3)

    print(f"Dev perplexity using FULL training data: {ppl_full:.2f}")
    print(f"Dev perplexity using HALF training data: {ppl_half:.2f}")


question1_half_training("1b_benchmark.train.tokens", "1b_benchmark.dev.tokens")


Dev perplexity using FULL training data: 7071.58
Dev perplexity using HALF training data: 8361.69


2..It would decrease the perplexity on unseen data. Because mapping more rare words to `<unk>` improves generalization and reduces perplexity, as the model learns more useful fallback behavior.

In [9]:
def question2_unk_threshold(train_path, dev_path, threshold_1=2, threshold_2=5):
    train_raw = load_sentences(train_path)
    dev_raw = load_sentences(dev_path)

    data1, vocab1 = handle_oov(train_raw, min_count=threshold_1)
    dev1 = [[w if w in vocab1 else "<unk>" for w in s] for s in dev_raw]
    model1 = train_laplace_model(data1, 3)
    ppl1 = evaluate_perplexity(model1, dev1, 3)

    data2, vocab2 = handle_oov(train_raw, min_count=threshold_2)
    dev2 = [[w if w in vocab2 else "<unk>" for w in s] for s in dev_raw]
    model2 = train_laplace_model(data2, 3)
    ppl2 = evaluate_perplexity(model2, dev2, 3)

    print(f"Dev perplexity with <unk> threshold {threshold_1}: {ppl1:.2f}")
    print(f"Dev perplexity with <unk> threshold {threshold_2}: {ppl2:.2f}")

# Example usage
question2_unk_threshold("1b_benchmark.train.tokens", "1b_benchmark.dev.tokens")


Dev perplexity with <unk> threshold 2: 10340.83
Dev perplexity with <unk> threshold 5: 4450.16
